In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import mpmath as mp
import math

### Get the repository path
import os
rep_path = os.getcwd().replace('/IPS', '')

import sys
sys.path.append(rep_path)
import python_functions as pf

### Import outputs from IPS code

In [ ]:
number_regions = 2

# Load the CSV files
points_real = pd.read_csv(f"/Users/erich/Downloads/Northeastern/ips_home/Data/ips_output/weierstrass/points_real_{number_regions}.csv", header=None).values
points_imag = pd.read_csv(f"/Users/erich/Downloads/Northeastern/ips_home/Data/ips_output/weierstrass/points_imag_{number_regions}.csv", header=None).values

# Load the rest
weights = pd.read_csv(f"/Users/erich/Downloads/Northeastern/ips_home/Data/ips_output/weierstrass/weights_{number_regions}.csv", header=None).values.flatten()
omegas = pd.read_csv(f"/Users/erich/Downloads/Northeastern/ips_home/Data/ips_output/weierstrass/omegas_{number_regions}.csv", header=None).values.flatten()
kappas = pd.read_csv(f"/Users/erich/Downloads/Northeastern/ips_home/Data/ips_output/weierstrass/kappas_{number_regions}.csv", header=None).values.flatten()

### Make an array that is similar to the samepled points from `cymetric`

In [ ]:
Z = points_real + 1j * points_imag

Z.shape

### Weierstrass coeficients

In [ ]:
a = 1.0
b = -4.0
c = 3.151212 * 60

w_coeffs = np.array([a, b, c])

### Plot 2D projection of the Weierstrass cubic

In [ ]:
# Keep the Z0!=0 patch and go to affine coordinates
mask = np.abs(Z[:,0]) > 1e-14
Z = Z[mask]
x = Z[:,1] / Z[:,0]
y = Z[:,2] / Z[:,0]

# Real 2D projection (paper-style square)
X = np.real(x)
Y = np.real(y)

# (Optional) clip extreme outliers for a tighter square view
q = 0.995
lo_x, hi_x = np.quantile(X, 1-q), np.quantile(X, q)
lo_y, hi_y = np.quantile(Y, 1-q), np.quantile(Y, q)

#plt.figure(figsize=(4,14))
plt.scatter(np.clip(X, lo_x, hi_x), np.clip(Y, lo_y, hi_y), c='k', s=1.0, alpha=0.8)
#plt.gca().set_aspect('equal', 'box')
plt.xlabel(r'Re$(Z_1/Z_0)$', fontsize = 15); plt.ylabel(r'Re$(Z_2/Z_0)$', fontsize = 15)
plt.xlim(lo_x, hi_x); plt.ylim(lo_y, hi_y)
#plt.title('Weierstrass cubic: affine (real) projection')
plt.tight_layout()
plt.show()

### Obtain uniformized coordiantes

In [ ]:
ab_x, ab_y = pf.weierstrass_uniform(Z = Z, weierstrass_coeff = w_coeffs)

ab_x.shape

### Plot the uniformized Weierstrass coordinates

In [ ]:
fig = plt.figure(figsize=(6, 6))

plt.scatter(ab_x, ab_y, c='k', s=2, alpha=0.7)

plt.gca().set_aspect('equal', 'box')

#fig.savefig(rep_path + "/IPS/uniform_weierstrass_cubic_ips.png", dpi=300, bbox_inches="tight")
plt.tight_layout(); plt.show()

### Colorize the different regions

In [ ]:
sampled_pts = ab_x.shape[0]
pts_per_reg = math.ceil(sampled_pts / number_regions)

fig = plt.figure(figsize=(6, 6))

for i in range(0, sampled_pts, pts_per_reg):

    plt.scatter(ab_x[i:i+pts_per_reg], ab_y[i:i+pts_per_reg], s=2, alpha=1)

    plt.gca().set_aspect('equal', 'box')

plt.tight_layout(); plt.show()

### Recreate Figure 3 from KL

In [ ]:
num_regs = np.array([1, 2, 3, 5, 11, 19])

fig, axes = plt.subplots(2, 3, figsize=(12, 8), constrained_layout=True)

for i in range(num_regs.shape[0]):
    
    reg = num_regs[i]
    points_real = pd.read_csv(f"/Users/erich/Downloads/Northeastern/ips_home/Data/ips_output/weierstrass/points_real_{reg}.csv", header=None).values
    points_imag = pd.read_csv(f"/Users/erich/Downloads/Northeastern/ips_home/Data/ips_output/weierstrass/points_imag_{reg}.csv", header=None).values
    
    Z = points_real + 1j * points_imag
    
    ab_x, ab_y = pf.weierstrass_uniform(Z = Z, weierstrass_coeff = w_coeffs)
    
    sampled_pts = ab_x.shape[0]
    pts_per_reg = math.ceil(sampled_pts / reg)
    
    for j in range(0, sampled_pts, pts_per_reg):
        
        axes.flat[i].scatter(ab_x[j:j+pts_per_reg], ab_y[j:j+pts_per_reg], s=2, alpha=0.7)
    
    axes.flat[i].set_aspect('equal', adjustable='box')
    axes.flat[i].set_xlim(-0.55, 0.55)
    axes.flat[i].set_ylim(-0.55, 0.55)
    axes.flat[i].tick_params(labelsize=8)

fig.savefig(rep_path + "/IPS/uniform_weierstrass_cubic_ips.png", dpi=300, bbox_inches="tight")

### Import data for Schoen Split Bicubic

In [ ]:
#points_real_bicubic = pd.read_csv("/Users/erich/Downloads/Northeastern/ips_home/Data/ips_output/bicubic/points_real_1.csv", header=None).values
#points_imag_bicubic = pd.read_csv("/Users/erich/Downloads/Northeastern/ips_home/Data/ips_output/bicubic/points_imag_1.csv", header=None).values

points_real_bicubic = np.load("/Users/erich/Downloads/Northeastern/ips_home/Data/ips_output/bicubic/points_real_1.npy")
points_imag_bicubic = np.load("/Users/erich/Downloads/Northeastern/ips_home/Data/ips_output/bicubic/points_imag_1.npy")

Z_bicubic = points_real_bicubic + 1j * points_imag_bicubic

Z_bicubic.shape

### Plot Schoen Split Bicubic

In [ ]:
fig, axes = pf.sampling_plot(Z_bicubic, q=0.995, equal_aspect=False)
plt.show()